In [6]:
def DelEmptyString(strlist):
    i = 0
    while i < len(strlist):
        if strlist[i] == None or len(strlist[i]) == 0:
            del strlist[i]
        else:
            i += 1
    return strlist

def ProcessPattern(ptxt):
    main_token = []
    constraints = []
    ### --- if ptxt == None or ptxt = "", return ---
    if not ptxt or len(ptxt.strip()) == 0:
        return main_token, constraints
    ### --- split pattern_txt into words & delete the empty words ----
    pwords = DelEmptyString(ptxt.split(' '))
    ### --- divide the words into main_tokens and constraints ---
    i = 0
    while i < len(pwords):
        temp_main = []
        temp_constraints = []
        ### ------- if current word is a constrain ----------
        while i < len(pwords):
            p = pwords[i]
            ### if current word start with '&', append it into temp constraints;
            if p[0] == '&':
                temp_constraints.append(p)
                i += 1
            ### if current word start with '(', find the whole ignore pieces, append it into temp constraints
            elif p[0] == '(':
                if p[-1] == ')':
                    temp_constraints.append(p)
                    i += 1
                else:
                    for j in range(i + 1, len(pwords)):
                        p = p + ' ' + pwords[j]
                        if pwords[j][-1] == ')':
                            temp_constraints.append(p)
                            i = j + 1
                            break
            else:
                break
        ### ------ if current word is a main_tokens --------
        while i < len(pwords):
            p = pwords[i]
            if p[0] not in ['(', '&']:
                ### ----- delete '/' ------
                if '/' in p:
                    temp_main.append(DelEmptyString(p.split('/')))
                else:
                    temp_main.append(p)
                i += 1
            else:
                break
        constraints.append(temp_constraints)
        if len(temp_main) > 0:
            main_token.append(temp_main)

    return main_token, constraints

In [56]:
main_token, constraints = ProcessPattern('&C (,/;/./--) consequently/test &THIS (,) &R')
#re.compile("[.?!\s]?([a-zA-z][a-zA-z\s]+[.?!\s]?) ([cC]onsequently), ([a-zA-z\s]+[.?!])")

In [57]:
main_token

[[['consequently', 'test']]]

In [58]:
constraints

[['&C', '(,/;/./--)'], ['&THIS', '(,)', '&R']]

In [54]:
import os
import nltk
import re
import operator
### 'MainTokenRegExp' intend to: create regular expression list for all patterns
### ---- in order to check whether the sentence contains a pattern's main_token ---
### ---- notice that the capture group (.*) is in accordance with Patterns constraints ----
def MainTokenRegExp(main_token, constraints):
    ### current Regular Expression for current patterns\
    ### if current pattern only have one main_token, find the first appearance of this main_token ----
    if len(main_token) == 1 and len(main_token[0]) == 1:
        curRegExp = r'(.*?)'
    ### else, try to match as much as the pattern can ---- 
    else:
        curRegExp = r'(.*)'
        ### patterns' main_token are broken into pieces, each piece must be matched successive.
    for pi in range(len(main_token)):
        ### ---- if current constraint pieces has a class constraint piece, add " " ----            
        for cp in constraints[pi]: #cp is string
            if cp[0] != '(': #TODO distinguish &C and &R
                curRegExp = curRegExp + r'[a-zA-z][a-zA-z\s]+'
                #curRegExp = curRegExp + r' ' !TODO
                break
            else: #if constraint has '()'
#                 curRegExp = curRegExp + cp
                continue #TODO 
                #case 1 (,/./?) char
                #case 2 (&AND) var
                #case 3 
        ### ---- add current main_token pieces into current Regular Expression -- curRegExp ----
        for ti in range(len(main_token[pi])):
            if ti > 0:
                curRegExp += r' '
            tokens = main_token[pi][ti]
            ### if current token is a string, add it into curRegExp directly
            if type(tokens) == str:
                if tokens in ['.', '?', ':']:
                    curRegExp += '\\'#.encode('utf-8') !! python3 return as bytes
                curRegExp += tokens#.encode('utf-8')
            ### if current token is a list, create a "no capture group" (?:token[0]|token[1]|...) for it
            #eg reason/reasons
            else:
                curRegExp = curRegExp + r'(?:'
                for tempt in range(len(tokens)):
                    if tempt > 0:
                        curRegExp = curRegExp + r'|'
                    if tokens[tempt] in ['.', '?', ':']:
                        curRegExp = curRegExp + '\\'#.encode('utf-8')
                    curRegExp = curRegExp + tokens[tempt]#.encode('utf-8')
                curRegExp = curRegExp + r')'
        ### ---- if pattern is ended with a main_token, stop add "(.*)" ----
        if len(main_token) == len(constraints) and pi == len(main_token) - 1:
            break
        ### ---- if next constraint pieces has a class constraint piece, add " " ----            
        for cp in constraints[pi + 1]:
            if cp[0] != '(':
                #curRegExp = curRegExp + r' ' TODO 
                curRegExp = curRegExp + r'[a-zA-z][a-zA-z\s]+'
                break
        ### ---- if current main_token pieces is not the last one, add (.*?), else add (.*) ----
        if pi < len(main_token) - 1:
            curRegExp = curRegExp + r'(.*?)'
        else:
            curRegExp = curRegExp + r'(.*)'

    return re.compile(curRegExp, re.I)  ###re.I means ignore upper or lower cases

#     Dumppickle(os.path.join(DICpkdir, 'mtRegExpList.pk'), mtRegExpList)

In [55]:
MainTokenRegExp(main_token, constraints)

re.compile(r'(.*?)[a-zA-z][a-zA-z\s]+(?:consequently|test)[a-zA-z][a-zA-z\s]+(.*)',
re.IGNORECASE|re.UNICODE)

In [ ]:
#manov script
import argparse
import re
import csv
#{&C (,/;/./--) consequently (,) &R}
def parse(input_file, output_file):
    txt = ''
    with open(input_file, 'r', encoding='utf-8') as f:
        txt = ''.join(f.readlines())
    
    ptrn = re.compile("[.?!\s]?([a-zA-z][a-zA-z\s]+[.?!\s]?) ([cC]onsequently), ([a-zA-z\s]+[.?!])")
    matches = ptrn.findall(txt)

    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Cause", "Connector", "Effect"])
        writer.writerows([[matches[0], matches[1], matches[2]] for matches in matches])


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Parsing arguments')
    parser.add_argument('--input', type=str, help='The path to the input file \
        or directory')
    parser.add_argument('--output', type=str, help='The path to the output file \
        or directory')
    
    args = parser.parse_args()
    
    parse(args.input, args.output)